In [15]:
from datasets import DatasetDict, Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    T5Config,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    GenerationConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
    pipeline,
)
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
import numpy as np
import tqdm
import nltk
import csv
import time
import gc
import GPUtil
import evaluate
import pprint
from numba import cuda
import optuna
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import wandb
import os
import pickle
from typing import Dict, Union, Optional, Tuple, List, Any

In [3]:
wandb.login()
os.environ["WAND_NOTEBOOK_NAME"] = "w266_final_project_models"
os.environ["WANDB_DIR"] = "../models/wandb"
os.environ["WANDB_PROJECT"] = "w266_final_project"

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garykong. Use `wandb login --relogin` to force relogin


In [4]:
# Random seed for reproducibility
RANDOM_SEED = 42

# Parameters for classification
BATCH_SIZE_EVAL = 32
BATCH_SIZE_TRAIN = 32

# Setting the DEVICE to cuda
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set path for profane word list
PROFANE_WORD_PATH = "../data/raw/en.txt"

# Set path for raw dataset dictionary
RAW_DATASET_PATH = "../data/processed/raw_dataset.pkl"

# Set maximum length for input and output
MAX_INPUT_LENGTH = 64
MAX_OUTPUT_LENGTH = 64

## Load dataset and get lengths

In [5]:
# Load dataset
raw_datasets = DatasetDict.load_from_disk(RAW_DATASET_PATH)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 10733
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 1193
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 671
    })
})

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

def get_lengths(raw_datasets, tokenizer):
    """
    Get the lengths of the source and target sequences in the dataset.

    Args:
        raw_datasets (DatasetDict): dictionary containing the raw dataset
        tokenizer (PreTrainedTokenizer): tokenizer to use for encoding the sequences
    
    Returns:
        source_lengths (list): list of lengths of source sequences
        target_lengths (list): list of lengths of target sequences
        max_source_length (int): maximum length of source sequence
        max_target_length (int): maximum length of target sequence
    """
    train_source_lengths = [len(tokenizer(x)["input_ids"]) for x in raw_datasets["train"]["source"]]
    val_source_lengths = [len(tokenizer(x)["input_ids"]) for x in raw_datasets["validation"]["source"]]
    test_source_lengths = [len(tokenizer(x)["input_ids"]) for x in raw_datasets["test"]["source"]]
    source_lengths = train_source_lengths + val_source_lengths + test_source_lengths
    max_source_length = max(source_lengths)

    train_target_lengths = [len(tokenizer(x)["input_ids"]) for x in raw_datasets["train"]["target"]]
    val_target_lengths = [len(tokenizer(x)["input_ids"]) for x in raw_datasets["validation"]["target"]]
    test_target_lengths = [len(tokenizer(x)["input_ids"]) for x in raw_datasets["test"]["target"]]
    target_lengths = train_target_lengths + val_target_lengths + test_target_lengths
    max_target_length = max(target_lengths)
    
    return source_lengths, target_lengths, max_source_length, max_target_length

def plot_lengths(source_lengths, target_lengths):
    """
    Plot the distribution of lengths of source and target sequences.

    Args:
        source_lengths (list): list of lengths of source sequences
        target_lengths (list): list of lengths of target sequences
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.histplot(source_lengths, ax=axes[0])
    sns.histplot(target_lengths, ax=axes[1])
    axes[0].set_title("Source Lengths")
    axes[1].set_title("Target Lengths")
    axes[0].set_xlabel("Length")
    axes[1].set_xlabel("Length")
    axes[0].set_ylabel("Count")
    axes[1].set_ylabel("Count")
    axes[0].set_xlim(0, 100)
    axes[1].set_xlim(0, 100)

# Get distribution of lengths of source across train, val, and test
bart_source_lengths, bart_target_lengths, bart_max_source_length, bart_max_target_length = get_lengths(raw_datasets, AutoTokenizer.from_pretrained("s-nlp/bart-base-detox"))

# Check what lengths would be after tokenization with t5
t5_source_lengths, t5_target_lengths, t5_max_source_length, t5_max_target_length = get_lengths(raw_datasets, AutoTokenizer.from_pretrained("t5-base"))

# Print max lengths
print("Maximum source length for BART:", bart_max_source_length)
print("Maximum target length for BART:", bart_max_target_length)
print("Maximum source length for T5:", t5_max_source_length)
print("Maximum target length for T5:", t5_max_target_length)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Maximum source length for BART: 38
Maximum target length for BART: 36
Maximum source length for T5: 47
Maximum target length for T5: 43


## Debugging Functions

In [6]:
def measure_time(func, *args, **kwargs):
    """
    Calculates the time it takes to run a function.
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Function {func.__name__} took {elapsed_time:.2f} seconds to run.")
    return result

def get_gpu_memory():
    """
    Gets the GPU memory information.
    """
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    print(f"Total GPU memory: {gpu.memoryTotal}MB")
    print(f"Free GPU memory: {gpu.memoryFree}MB")
    print(f"Used GPU memory: {gpu.memoryUsed}MB")

def force_clear_GPU_memory():
    """
    Force clears the GPU memory.
    """
    cuda.select_device(0)
    cuda.close()

def cleanup():
    """
    Cleans up the GPU memory.
    """
    gc.collect()
    torch.cuda.empty_cache()

## Evaluation Metrics

In [7]:
# Initialize model variables
model_bleurt = None
model_bertscore = None
model_sacrebleu = None

# Toxicity classifier
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained(
    "SkolkovoInstitute/roberta_toxicity_classifier"
)

# Acceptability classifier
tokenizer_acceptability = AutoTokenizer.from_pretrained("iproskurina/tda-bert-en-cola")
model_acceptability = AutoModelForSequenceClassification.from_pretrained("iproskurina/tda-bert-en-cola")

def calc_sacrebleu(refs, preds):
    """
    Calculates the SacreBLEU score.

    Args:
        refs (list): List of reference sentences
        preds (list): List of predicted sentences
    
    Returns:
        results (float): SacreBLEU score
    """
    global model_sacrebleu

    if model_sacrebleu is None:
        model_sacrebleu = evaluate.load("sacrebleu")

    results = model_sacrebleu.compute(predictions=preds, references=refs)["score"]
    results = results/100

    return results

def calc_bert_score(
    refs, preds, model_type="microsoft/deberta-large-mnli", output_mean=True
    ):
    """
    Calculates BERT score per line. Note: https://docs.google.com/spreadsheets/d/1RKOVpselB98Nnh_EOC4A2BYn8_201tmPODpNWu4w7xI/edit#gid=0 lists the best performing models
    Args:
        refs (list): List of reference sentences.
        y_pred (list): List of predicted sentences.
        model_type (str): Type of BERT model to use.
        output_mean (bool): Whether to output the mean of the scores.

    Returns:
        list of precision, recall, f1 scores.

    """
    global model_bertscore

    if model_bertscore is None:
        model_bertscore = evaluate.load("bertscore")
        
    results = model_bertscore.compute(predictions=preds, references=refs, model_type=model_type)
    precision = np.array(results["precision"])
    recall = np.array(results["recall"])
    f1 = np.array(results["f1"])
    
    if output_mean:
        precision = precision.mean()
        recall = recall.mean()
        f1 = f1.mean()

    return precision, recall, f1

def calc_bleurt(refs, preds, checkpoint="BLEURT-20_D12", output_mean = True):
    """
    Calculates BLEURT score per line.

    Args:
        refs (list): List of reference sentences.
        preds (list): List of predicted sentences.
        output_type (str): Type of output to return. Either 'numpy' or 'list'.

    Returns:
        list/array of BLEURT scores.
    """
    global model_bleurt

    if model_bleurt is None:
        model_bleurt = evaluate.load("bleurt", module_type="metric", checkpoint=checkpoint)

    results = np.array(model_bleurt.compute(predictions=preds, references=refs)["scores"])

    if output_mean:
        results = results.mean()

    return results

def calc_tox_acceptability(
    data,
    tokenizer,
    model,
    output_score=True,
    output_mean=True):
    """
    Calculates toxicity and acceptability scores for a given dataset.

    Args:
        data = list of strings to be evaluated
        tokenizer = tokenizer for the model
        model = model to be used for evaluation
        output_score = whether to output the score or the label
        output_mean = whether to output the mean of the scores or the scores for each sentence
    
    Returns:
        array of toxicity and acceptability scores.
    """
    model = model.to(DEVICE)
    
    inputs = tokenizer(data, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs)["logits"]
        if output_score:
            result = torch.nn.functional.softmax(logits, dim=1)[:, 1]
        else:
            result = logits.argmax(1).data
        result = result.cpu().numpy()

    if output_mean:
        result = result.mean()
        
    return result

def evaluate_metrics(
    refs,
    preds,
    tokenizer_toxicity=tokenizer_toxicity,
    model_toxicity=model_toxicity,
    tokenizer_acceptability=tokenizer_acceptability,
    model_acceptability=model_acceptability,
    to_neutral=True,
    weights={
        "BLEU": 0.2,
        "STA": 0.4,
        "Acceptability": 0.2,
        "BERT_Score": 0.2
    },
    include_bleurt=False
):
    """
    Calculates and returns a dictionary of evaluation metrics

    Args:
        refs (list): list of strings (reference)
        preds (list): list of strings (predictions)
        tokenizer_toxicity (tokenizer): tokenizer for toxicity model
        model_toxicity (model): toxicity model
        tokenizer_acceptability (tokenizer): tokenizer for acceptability model
        model_acceptability (model): acceptability model
        to_neutral (bool): whether the goal is to transfer to neutral (True) or to toxic (False)
        weights (dict): dictionary of weights for each metric
        include_bleurt (bool): whether to include BLEURT score in the output

    Returns:
        results (dict): dictionary of evaluation metrics
    """

    # Calculate BLEU score
    bleu = calc_sacrebleu(refs, preds)

    # Calculate toxicity classification
    # tox_ref = calc_tox_acceptability(refs, tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)
    tox_pred = calc_tox_acceptability(preds, tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)

    # Calculate style transfer accuracy as proportion of sentences that were correctly classified (as non-toxic / toxic)
    if to_neutral:
        sta_correct_label = 0
    else:
        sta_correct_label = 1

    # sta_ref = (tox_ref == sta_correct_label).sum() / len(tox_ref)
    sta_pred = (tox_pred == sta_correct_label).sum() / len(tox_pred)
    # sta_pct = sta_pred / sta_ref

    # Calculate acceptability scores
    # acc_ref = calc_tox_acceptability(refs, tokenizer_acceptability, model_acceptability)
    acc_pred = calc_tox_acceptability(preds, tokenizer_acceptability, model_acceptability)
    # acc_pct = acc_pred / acc_ref

    # Calculate similarity score
    bert_score_f1 = calc_bert_score(refs, preds, model_type="distilbert-base-uncased")[2]

    # Calculate BLEURT score if include_bleurt is True
    bleurt = None
    if include_bleurt:
        bleurt = calc_bleurt(refs, preds)

    # Calculate composite score
    composite_score = weights["BLEU"] * bleu + weights["STA"] * sta_pred + weights["Acceptability"] * acc_pred + weights["BERT_Score"] * bert_score_f1

    # Return a dictionary of metrics
    results = {
        "BLEU": bleu,
        "STA_preds": sta_pred,
        # "STA_pct": sta_pct,
        "Acceptability_preds": acc_pred,
        # "Acceptability_pct": acc_pct,
        "BERT_score_f1": bert_score_f1,
        "Overall": composite_score,
    }
    if include_bleurt:
        results["BLEURT"] = bleurt
        
    return results


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Baseline Model (DELETE)

In [9]:
def baseline_detoxifier(text_list, profane_word_path=PROFANE_WORD_PATH):
    """
    Returns a detoxified version of the text by replacing toxic terms with blanks

    Args:
        text_list (list): list of strings to be detoxified
        toxic_list (list): list of toxic terms to be removed from text_list

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    # Load list of profane words
    profane_words = []
    with open(profane_word_path, "r") as f:
        for line in f:
            profane_words.append(line.strip())

    # Detoxify text
    detoxified_text_list = []
    for text in text_list:
        for term in profane_words:
            text = text.replace(term, "")
        detoxified_text_list.append(text)

    return detoxified_text_list

y_pred_delete = baseline_detoxifier(raw_datasets["validation"]['source'])
delete_eval_validation = evaluate_metrics(raw_datasets["validation"]['target'], y_pred_delete)
delete_eval_validation

{'BLEU': 0.5291006187073797,
 'STA_preds': 0.6596814752724225,
 'Acceptability_preds': 0.47865131,
 'BERT_score_f1': 0.9118211825455524,
 'Overall': 0.6477872133543217}

# Baseline Model (BART Base Detox)

In [ ]:
Y_PRED_BART_PICKLE_FILE = "../data/processed/y_pred_bart.pkl"

if os.path.isfile(Y_PRED_BART_PICKLE_FILE):
    # Load predictions from pickle file
    with open(Y_PRED_BART_PICKLE_FILE, "rb") as f:
        y_pred_bart = pickle.load(f)
else:
    # Create predictions using BART
    pipe_bart = pipeline("text2text-generation", model="s-nlp/bart-base-detox", device=DEVICE)

    # Create predictions using BART and show progress using tqdm
    y_pred_bart = pipe_bart(raw_datasets["validation"]['source'], max_length=MAX_OUTPUT_LENGTH, truncation=True)

    # Convert to list of strings
    y_pred_bart = [x["generated_text"] for x in y_pred_bart]
    y_pred_bart[:5]

    # Save predictions as a pickle file
    with open(Y_PRED_BART_PICKLE_FILE, "wb") as f:
        pickle.dump(y_pred_bart, f)

In [ ]:
# Evaluate BART predictions
bart_eval_validation = evaluate_metrics(raw_datasets["validation"]['target'], y_pred_bart)
bart_eval_validation

{'BLEU': 0.7015951162845684,
 'STA_preds': 0.9178541492036881,
 'Acceptability_preds': 0.71802455,
 'BERT_score_f1': 0.9451393306205379,
 'Overall': 0.8400934594361843}

# Helper Functions to Fine-tune T5 Models

In [8]:
model_t5_base = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer_t5_base = T5Tokenizer.from_pretrained("t5-base")
model_t5_small = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer_t5_small = T5Tokenizer.from_pretrained("t5-small")

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [9]:
def add_prefix(datasetdict, prefix="to_neutral: "):
    """Adds a prefix to the source sequence in the dataset."""
    datasetdict_copy = datasetdict.copy()
    datasetdict_copy["train"] = datasetdict_copy["train"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["validation"] = datasetdict_copy["validation"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["test"] = datasetdict_copy["test"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy = DatasetDict(datasetdict_copy)
    return datasetdict_copy

def preprocess_function(examples, tokenizer):
    """Preprocess function for T5."""
    model_inputs = tokenizer(
        examples["source"],
        text_target=examples["target"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
    )
    return model_inputs

def post_process(preds, refs, tokenizer):
    """
    Post-process function for T5.

    Args:
        preds (list): list of predicted sequences
        refs (list): list of reference sequences
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding

    Returns:
        decoded_preds (list): list of decoded predicted sequences
        decoded_refs (list): list of decoded reference sequences
    """
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_refs = [ref.strip() for ref in decoded_refs]

    return decoded_preds, decoded_refs

def compute_metrics(eval_preds, tokenizer):
    """
    Function to calculate the metrics for trainer.evaluate().

    Args:
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        eval_preds (tuple): Tuple containing the predictions and references

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # Post-process the predictions and references
    decoded_preds, decoded_refs = post_process(preds, refs, tokenizer)
    
    # Evaluate metrics
    return evaluate_metrics(
        decoded_refs,
        decoded_preds,
        tokenizer_toxicity=tokenizer_toxicity,
        model_toxicity=model_toxicity,
        tokenizer_acceptability=tokenizer_acceptability,
        model_acceptability=model_acceptability,
        include_bleurt=False
    )

def setup_trainer(output_dir_name,
                train_dataset,
                eval_dataset,
                model_checkpoint="t5-small",
                per_device_train_batch_size=32,
                per_device_eval_batch_size=128,
                learning_rate=1e-4,
                num_train_epochs=30,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=4,
                compute_metrics=compute_metrics,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]):
    """
    Set up a Seq2SeqTrainer object for training a T5 model.

    Default parameters based on this: https://github.com/google-research/text-to-text-transfer-transformer/blob/main/t5/models/hf_model.py#L55

    Args:
        output_dir_name (str): What to name the model in the output directory.
        model_checkpoint (str): Name of the pre-trained model to use.
        train_dataset (Dataset): Training dataset.
        eval_dataset (Dataset): Validation/test dataset.
        per_device_train_batch_size (int): Batch size for training.
        per_device_eval_batch_size (int): Batch size for evaluation.
        learning_rate (float): Learning rate for optimizer.
        weight_decay (float): Weight decay for optimizer.
        num_train_epochs (int): Number of training epochs.
        max_length (int): Maximum length of generated sequences.
        num_beams (int): Number of beams for beam search.
        compute_metrics (function): Function to compute evaluation metrics.

    Returns:
        Seq2SeqTrainer: Trainer object for training the T5 model.
    """
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    GEN_CONFIG_PATH = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(GEN_CONFIG_PATH)

    # Define the training arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=GEN_CONFIG_PATH,
        fp16=True,
        report_to="wandb",
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
    )

    # Create a partial function with the tokenizer argument included
    compute_metrics_with_tokenizer = partial(compute_metrics, tokenizer=tokenizer)
    
    # Instantiate the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_with_tokenizer,
        callbacks=callbacks
    )

    return trainer

# Fine-tune T5 (Unidirectional)

### T5-Small

In [15]:
prefixed_datasets = add_prefix(raw_datasets)

tokenized_datasets_t5_small = prefixed_datasets.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small = setup_trainer(
    output_dir_name="t5-small-detoxify",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["validation"],
)

Map:   0%|          | 0/10733 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

In [ ]:
wandb.init(project="w266_final_project", name="t5-small-detoxify")
trainer_t5_small.train() # General rule is 10% number of epochs
wandb.finish()

Map:   0%|          | 0/10733 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta Preds,Acceptability Preds,Bert Score F1,Overall
1,1.250000,1.002992,0.577241,0.743504,0.678014,0.921097,0.732672
2,1.062000,0.954667,0.592385,0.821459,0.688148,0.924167,0.769524
3,0.995800,0.939699,0.598228,0.842414,0.690134,0.925105,0.779659
4,0.948300,0.923069,0.597996,0.854987,0.694469,0.924770,0.785442
5,0.913600,0.915044,0.601244,0.865046,0.701344,0.925471,0.791630
6,0.882700,0.918080,0.606649,0.891031,0.699932,0.925584,0.802845
7,0.855500,0.922288,0.600217,0.873428,0.698467,0.926352,0.794379
8,0.829700,0.919788,0.601931,0.894384,0.696203,0.925716,0.802524


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Acceptability_preds,▁▄▅▆██▇▆
eval/BERT_score_f1,▁▅▆▆▇▇█▇
eval/BLEU,▁▅▆▆▇█▆▇
eval/Overall,▁▅▆▆▇█▇█
eval/STA_preds,▁▅▆▆▇█▇█
eval/loss,█▄▃▂▁▁▂▁
eval/runtime,▇▇▁▄▇▂▆█
eval/samples_per_second,▂▂█▄▂▇▃▁
eval/steps_per_second,▂▂█▅▂▇▃▁
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███


#### Investigate difference between checkpoints

Interestingly, overall scores are maximized even as validation loss increases (see t5-small-detoxify). We can see that the main aspect that improves is style transfer accuracy (STA), but also acceptability. Next, we check samples of text generated at the checkpoint in which validation loss is minimized vs. the checkpoint where the overall score is maximized and compare them

In [10]:
# Define checkpoint paths
CHECKPOINT_T5_SMALL_MINLOSS = "../models/t5-small-detoxify/checkpoint-1680" # Epoch 5, min loss
CHECKPOINT_T5_SMALL_BEST = "../models/t5-small-detoxify/checkpoint-2016" # Epoch 15, best overall

In [16]:
# The trainer object for t5-small-detoxify is already the best model, so we can just load it
trainer_t5_small_best = setup_trainer(
    output_dir_name="t5-small-detoxify",
    model_checkpoint=CHECKPOINT_T5_SMALL_BEST,
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["validation"],
)

# Load the trainer object for t5-small-detoxify with the minimum loss
trainer_t5_small_minloss = setup_trainer(
    output_dir_name="t5-small-detoxify",
    model_checkpoint=CHECKPOINT_T5_SMALL_MINLOSS,
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["validation"],
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
# Get predictions from trainer objects
def get_preds_df (trainer_1=trainer_t5_small_best,
                  trainer_2=trainer_t5_small_minloss,
                  tokenizer=tokenizer_t5_small,
                  tokenized_data=tokenized_datasets_t5_small["validation"],
                  raw_data=raw_datasets["validation"]):
    """
    Gets the predictions from a trainer object and returns a dataframe containing the source, target, and prediction.

    Args:
        trainer_object (Seq2SeqTrainer): trainer object containing the model
        tokenized_data (Dataset): dataset to be evaluated
        raw_data (Dataset): dataset containing the raw data

    Returns:
        preds (Seq2SeqPrediction): Seq2SeqPrediction object containing the predictions
        decoded_preds (list): list of decoded predictions
        df (DataFrame): dataframe containing the source, target, and prediction
    """
    
    # Get predictions
    preds_1 = trainer_1.predict(tokenized_data)
    decoded_preds_1 = tokenizer.batch_decode(preds_1.predictions, skip_special_tokens=True)
    preds_2 = trainer_2.predict(tokenized_data)
    decoded_preds_2 = tokenizer.batch_decode(preds_2.predictions, skip_special_tokens=True)

    # Create dataframe containing source, target, and prediction
    df = pd.DataFrame()
    df["source"] = raw_data["source"]
    df["target"] = raw_data["target"]
    df["pred_1"] = decoded_preds_1
    df["pred_2"] = decoded_preds_2
    
    return df

# Get predictions for t5-small-detoxify
t5_small_df = get_preds_df(trainer_1=trainer_t5_small_best,
                                trainer_2=trainer_t5_small_minloss,
                                tokenizer=tokenizer_t5_small,
                                tokenized_data=tokenized_datasets_t5_small["validation"],
                                raw_data=raw_datasets["validation"])

In [18]:
# # Calculate relevant metric columns for t5_small_df
# t5_small_df["pred_1_BLEURT"] = calc_bleurt(t5_small_df["target"], t5_small_df["pred_1"], output_mean=False)
# t5_small_df["pred_2_BLEURT"] = calc_bleurt(t5_small_df["target"], t5_small_df["pred_2"], output_mean=False)
t5_small_df["pred_1_toxic_class"] = calc_tox_acceptability(t5_small_df["pred_1"].tolist(), tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)
t5_small_df["pred_2_toxic_class"] = calc_tox_acceptability(t5_small_df["pred_2"].tolist(), tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)
t5_small_df["pred_1_toxic_score"] = calc_tox_acceptability(t5_small_df["pred_1"].tolist(), tokenizer_toxicity, model_toxicity, output_score=True, output_mean=False)
t5_small_df["pred_2_toxic_score"] = calc_tox_acceptability(t5_small_df["pred_2"].tolist(), tokenizer_toxicity, model_toxicity, output_score=True, output_mean=False)
t5_small_df["source_acceptability"] = calc_tox_acceptability(t5_small_df["source"].tolist(), tokenizer_acceptability, model_acceptability, output_score=True, output_mean=False)
t5_small_df["target_acceptability"] = calc_tox_acceptability(t5_small_df["target"].tolist(), tokenizer_acceptability, model_acceptability, output_score=True, output_mean=False)
t5_small_df["pred_1_acceptability"] = calc_tox_acceptability(t5_small_df["pred_1"].tolist(), tokenizer_acceptability, model_acceptability, output_score=True, output_mean=False)
t5_small_df["pred_2_acceptability"] = calc_tox_acceptability(t5_small_df["pred_2"].tolist(), tokenizer_acceptability, model_acceptability, output_score=True, output_mean=False)
t5_small_df["pred_1_BERT_score"] = calc_bert_score(t5_small_df["target"], t5_small_df["pred_1"], model_type="distilbert-base-uncased", output_mean=False)[2]
t5_small_df["pred_2_BERT_score"] = calc_bert_score(t5_small_df["target"], t5_small_df["pred_2"], model_type="distilbert-base-uncased", output_mean=False)[2]

# Calculate differences in BLEURT, toxicity, acceptability, and BERT score
t5_small_df["diff_toxic_score"] = t5_small_df["pred_1_toxic_score"] - t5_small_df["pred_2_toxic_score"]
# t5_small_df["diff_BLEURT"] = t5_small_df["pred_1_BLEURT"] - t5_small_df["pred_2_BLEURT"]
t5_small_df["diff_acceptability"] = t5_small_df["pred_1_acceptability"] - t5_small_df["pred_2_acceptability"]
t5_small_df["diff_BERT_score"] = t5_small_df["pred_1_BERT_score"] - t5_small_df["pred_2_BERT_score"]

# Filter to rows where pred_1_toxicity is less than pred_2_toxicity
t5_small_df_filtered = t5_small_df[t5_small_df["pred_1_toxic_class"] < t5_small_df["pred_2_toxic_class"]]

# Print as individual lines
for index, row in t5_small_df_filtered.head(10).iterrows():
    print("Index:", index)
    print("Source:", row["source"])
    print("Target:", row["target"])
    print("Prediction (Best model):", row["pred_1"])
    print("Prediction (Min val error):", row["pred_2"])
    print("Diff in toxicity:", round(row["diff_toxic_score"], 3))
    # print("Diff in BLEURT:", round(row["diff_BLEURT"], 3))
    print("Diff in acceptability:", round(row["diff_acceptability"], 3))
    print("Diff in BERT:", round(row["diff_BERT_score"], 3))
    print()

Index: 62
Source: Spank them and send them back to california.
Target: Scold them and send them back to california
Prediction (Best model): Send them back to california
Prediction (Min val error): Spank them and send them back to California.
Diff in toxicity: -0.994
Diff in acceptability: -0.006
Diff in BERT: -0.002

Index: 84
Source: Its the adults who are screaming and carrying on like idiots.
Target: Its the adults who are screaming and carrying on
Prediction (Best model): It's the adults who are screaming and carrying on.
Prediction (Min val error): Its the adults who are screaming and carrying on like idiots.
Diff in toxicity: -0.999
Diff in acceptability: 0.699
Diff in BERT: -0.013

Index: 132
Source: All canadians are paying for what the left wing voted in, many canadians were not so stupid.
Target: All canadians are paying for what the left wing voted in, many canadians had common sense.
Prediction (Best model): All canadians are paying for what the left wing voted in, many can

Based on this sample, it looks like the predictions from the best model are indeed better at detoxifying text (exmple 108, 211) and is sufficiently different from the target text that there is no indication of overfitting per se. So going forward we prioritize selecting best models based on the overall score as opposed to minimizing validation loss.

## T5-Base

In [20]:
prefixed_datasets = add_prefix(raw_datasets)

tokenized_datasets_t5_base = prefixed_datasets.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_base},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_base = setup_trainer(
    output_dir_name="t5-base-detoxify",
    model_checkpoint="t5-base",
    train_dataset=tokenized_datasets_t5_base["train"],
    eval_dataset=tokenized_datasets_t5_base["validation"],
)

wandb.init(project="w266_final_project", name="t5-base-detoxify")
trainer_t5_base.train()
wandb.finish()

Map:   0%|          | 0/10733 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Sta Preds,Acceptability Preds,Bert Score F1,Overall
1,nan,0.812222,0.608180,0.890193,0.723307,0.925496,0.807474
2,nan,0.798929,0.601930,0.883487,0.709482,0.926611,0.800999
3,nan,0.798679,0.608734,0.906119,0.720927,0.926683,0.813716
4,nan,0.807012,0.605299,0.899413,0.716527,0.927316,0.809594
5,nan,0.830957,0.609381,0.911148,0.716312,0.927588,0.815116
6,nan,0.846663,0.604496,0.909472,0.720942,0.926022,0.814081
7,nan,0.883831,0.602264,0.907795,0.720766,0.926995,0.813123


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Acceptability_preds,█▁▇▅▄▇▇
eval/BERT_score_f1,▁▅▅▇█▃▆
eval/BLEU,▇▁▇▄█▃▁
eval/Overall,▄▁▇▅█▇▇
eval/STA_preds,▃▁▇▅██▇
eval/loss,▂▁▁▂▄▅█
eval/runtime,▇▆█▁▆▁▆
eval/samples_per_second,▂▃▁█▃█▃
eval/steps_per_second,▂▃▁█▃█▃
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███


It seems like t5-small and t5-base performs similarly, but t5-base reaches convergence much earlier. There's also indication of overfitting occurring much earlier (e.g., by epoch 5 validation loss jumps up significantly) without overall performance improving. We will use t5-small going forward given computational constraints. Similar performance may indicate that the task at hand is not too complex and the extra complexity of t5-base is not needed.

# Fine-tune T5 Model (Bi-directional, No custom loss)

### Helper Functions

In [20]:
def create_bidirectional_dataset(datasets):
    """
    Creates a bi-directional dataset from the original dataset.

    Args:
        datasets (DatasetDict): DatasetDict object containing the original dataset.
    
    Returns:
        extended_datasets (DatasetDict): DatasetDict object containing the bi-directional dataset.
    """

    def bidirectional_extension(dataset):
        new_data = {
            "source": [],
            "target": []
        }
        for src, tgt in zip(dataset['source'], dataset['target']):
            new_data['source'].extend([f'to_neutral: {src}', f'to_toxic: {tgt}'])
            new_data['target'].extend([tgt, src])
        return new_data

    extended_train_data = bidirectional_extension(datasets["train"])
    extended_validation_data = bidirectional_extension(datasets["validation"])
    extended_test_data = bidirectional_extension(datasets["test"])

    extended_datasets = DatasetDict({
        "train": Dataset.from_dict(extended_train_data),
        "validation": Dataset.from_dict(extended_validation_data),
        "test": Dataset.from_dict(extended_test_data)
    })

    return extended_datasets

def get_indices(dataset):
    """
    Saves the indices of data that is to_neutral and to_toxic.
    """
    to_neutral_idx = []
    to_toxic_idx = []
    for i in range(len(dataset)):
        if dataset[i]["source"].startswith("to_neutral"):
            to_neutral_idx.append(i)
        else:
            to_toxic_idx.append(i)

    return to_neutral_idx, to_toxic_idx

def compute_metrics_bd(eval_preds, tokenizer, bd_dataset, shuffled_data=False):
    """
    Function to calculate the metrics for trainer.evaluate().
    This function is for the bi-directional model.
    
    Args:
        eval_preds (tuple): Tuple containing the predictions and references
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        shuffled_data (bool): Whether the data is shuffled or not
        bd_dataset (DatasetDict): Bidirectional dataset created using create_bidirectional_datasets e.g., raw_datasets_bd["validation"]

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_refs = [ref.strip() for ref in decoded_refs]
    
    # If shuffled data is false, have to_neutral_preds and to_neutral_refs just be predictions and refs with even indices
    if not shuffled_data:
        to_neutral_preds = decoded_preds[::2]
        to_neutral_refs = decoded_refs[::2]
    # Otherwise, get the indices to use when splitting predictions and refs to to_neutral and to_toxic
    else:
        # Get the indices to use when splitting predictions and refs to to_neutral and to_toxic
        to_neutral_idx = get_indices(bd_dataset)[0]

        # Retrieve based on the indices
        to_neutral_preds = [decoded_preds[i] for i in to_neutral_idx]
        to_neutral_refs = [decoded_refs[i] for i in to_neutral_idx]
    
    # Evaluate metrics for to_neutral
    to_neutral_metrics = evaluate_metrics(
        to_neutral_refs,
        to_neutral_preds,
        to_neutral=True
    )

    # Return dictionary of to_neutral metrics
    return to_neutral_metrics

### Trial without shuffled data

In [21]:
raw_datasets_bd = create_bidirectional_dataset(raw_datasets)

tokenized_datasets_bd_t5_small = raw_datasets_bd.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_bd = setup_trainer(
    output_dir_name="t5-small-detoxify-bd-noshuffle",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_bd_t5_small["train"],
    eval_dataset=tokenized_datasets_bd_t5_small["validation"],
    compute_metrics=partial(compute_metrics_bd, bd_dataset=raw_datasets_bd["validation"], shuffled_data=False)
    )

wandb.init(project="w266_final_project", name="t5-small-bd-noshuffle-detoxify")
trainer_t5_small_bd.train()
wandb.finish()

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2386 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta Preds,Acceptability Preds,Bert Score F1,Overall
1,1.581200,1.297624,0.582431,0.710813,0.678005,0.919003,0.720213
2,1.343000,1.220252,0.595135,0.795474,0.682763,0.922740,0.758317
3,1.257800,1.186172,0.601159,0.863370,0.692295,0.924668,0.788972
4,1.203900,1.174466,0.602681,0.865884,0.687537,0.924235,0.789244
5,1.166600,1.154176,0.601517,0.883487,0.702484,0.925840,0.799363
6,1.131100,1.153290,0.607493,0.900251,0.701396,0.925796,0.807038
7,1.104700,1.140285,0.607121,0.891869,0.704214,0.926410,0.804297
8,1.079200,1.136118,0.606703,0.893546,0.697141,0.927136,0.803614


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Acceptability_preds,▁▂▅▄█▇█▆
eval/BERT_score_f1,▁▄▆▆▇▇▇█
eval/BLEU,▁▅▆▇▆███
eval/Overall,▁▄▇▇▇███
eval/STA_preds,▁▄▇▇▇███
eval/loss,█▅▃▃▂▂▁▁
eval/runtime,▁▂▅▄█▆▅▇
eval/samples_per_second,█▇▄▅▁▃▄▂
eval/steps_per_second,█▇▄▅▁▃▄▂
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███


### Trial with shuffled data

In [23]:
raw_datasets_bd_shuffle = raw_datasets_bd.copy()
raw_datasets_bd_shuffle["train"] = raw_datasets_bd_shuffle["train"].shuffle(seed=RANDOM_SEED)
raw_datasets_bd_shuffle = DatasetDict(raw_datasets_bd_shuffle)

tokenized_datasets_bd_shuffle_t5_small = raw_datasets_bd_shuffle.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_bd_shuffle = setup_trainer(
    output_dir_name="t5-small-detoxify-bd-shuffle",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_bd_shuffle_t5_small["train"],
    eval_dataset=tokenized_datasets_bd_shuffle_t5_small["validation"],
    compute_metrics=partial(compute_metrics_bd, bd_dataset=raw_datasets_bd["validation"], shuffled_data=True)
    )

wandb.init(project="w266_final_project", name="t5-small-bd-shuffle-detoxify")
trainer_t5_small_bd_shuffle.train()
wandb.finish()

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2386 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta Preds,Acceptability Preds,Bert Score F1,Overall
1,1.582300,1.291271,0.582313,0.698240,0.689065,0.918604,0.717292
2,1.340700,1.219955,0.596120,0.811400,0.687961,0.922587,0.765893
3,1.256100,1.183220,0.604068,0.852473,0.694322,0.924519,0.785571
4,1.204600,1.167809,0.602693,0.865884,0.698694,0.924281,0.791487
5,1.161100,1.155576,0.602983,0.865046,0.691605,0.925625,0.790061
6,1.128800,1.152056,0.603047,0.885163,0.707908,0.925451,0.801347
7,1.099900,1.140844,0.606081,0.906957,0.707975,0.925890,0.810772
8,1.075200,1.137721,0.605489,0.901090,0.701951,0.926568,0.807237
9,1.053600,1.131585,0.607370,0.904443,0.704556,0.926540,0.809470


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Acceptability_preds,▁▁▃▅▂██▆▇
eval/BERT_score_f1,▁▅▆▆▇▇▇██
eval/BLEU,▁▅▇▇▇▇█▇█
eval/Overall,▁▅▆▇▆▇███
eval/STA_preds,▁▅▆▇▇▇███
eval/loss,█▅▃▃▂▂▁▁▁
eval/runtime,▁▂▂▇██▇▆▆
eval/samples_per_second,█▇▇▂▁▁▂▃▃
eval/steps_per_second,█▇▇▂▂▁▂▂▃
train/epoch,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███


In [ ]:
# Need to check that there isn't a bug in sta_pct (seems odd that it's so high)

# Fine-tune T5 Model (Uni-directional, with Custom Loss)

Documentation for original loss function:

https://huggingface.co/transformers/v4.2.2/_modules/transformers/trainer.html#Trainer.compute_loss

In [ ]:
# def compute_loss(self, model, inputs):
#         """
#         How the loss is computed by Trainer. By default, all models return the loss in the first element.

#         Subclass and override for custom behavior.
#         """
#         outputs = model(**inputs)
#         # Save past state if it exists
#         # TODO: this needs to be fixed and made cleaner later.
#         if self.args.past_index >= 0:
#             self._past = outputs[self.args.past_index]

#         if self.label_smoother is not None and "labels" in inputs:
#             return self.label_smoother(outputs, inputs["labels"])
#         else:
#             # We don't use .loss here since the model may return tuples instead of ModelOutput.
#             return outputs["loss"] if isinstance(outputs, dict) else outputs[0]

## Classifier-guided loss

First we test the classifier-guided loss

In [38]:
# Constants
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model for toxicity classification
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained(
    "SkolkovoInstitute/roberta_toxicity_classifier"
).to(DEVICE)

# Mock class
class MockClass:
    def __init__(self):
        self.tokenizer_toxicity = tokenizer_toxicity
        self.model_toxicity = model_toxicity

    def classifier_guided_loss(self, generated_text, target_label):
        print("Calculating classifier-guided loss...")
        inputs = self.tokenizer_toxicity(generated_text, return_tensors="pt", padding=True).to(DEVICE)
        batch_size = inputs["input_ids"].shape[0]
        target_label = torch.tensor([target_label]*batch_size, dtype=torch.long).to(DEVICE)   
        with torch.no_grad():
            logits = self.model_toxicity(**inputs)["logits"]
        print(f"Logits: {logits}")
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits, target_label)
        print(f"Classifier-guided loss: {loss}")
        return loss

# Test the classifier_guided_loss method with a list of generated text
target_label_neutral = 0

# Test Case 1: All Toxic Texts
generated_texts_toxic = ["You are stupid.", "Go to hell.", "You're an idiot."]
print("Test Case 1: All Toxic Texts")
loss_toxic = mock_instance.classifier_guided_loss(generated_texts_toxic, target_label_neutral)
print(f"Loss for all toxic texts: {loss_toxic}\n")

# Test Case 2: All Neutral Texts
generated_texts_neutral = ["You are smart.", "Have a nice day.", "You're intelligent."]
print("Test Case 2: All Neutral Texts")
loss_neutral = mock_instance.classifier_guided_loss(generated_texts_neutral, target_label_neutral)
print(f"Loss for all neutral texts: {loss_neutral}\n")

# Test Case 3: Mixed Texts
generated_texts_mixed = ["You are stupid.", "Have a nice day.", "You are so nice", "You're an idiot."]
print("Test Case 3: Mixed Texts")
loss_mixed = mock_instance.classifier_guided_loss(generated_texts_mixed, target_label_neutral)
print(f"Loss for mixed texts: {loss_mixed}\n")

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test Case 1: All Toxic Texts
Calculating classifier-guided loss...
Logits: tensor([[-3.9838,  3.9090],
        [-3.5358,  3.3512],
        [-3.9934,  3.9302]], device='cuda:0')
Classifier-guided loss: 7.5684027671813965
Loss for all toxic texts: 7.5684027671813965

Test Case 2: All Neutral Texts
Calculating classifier-guided loss...
Logits: tensor([[ 4.1200, -4.0838],
        [ 4.8206, -5.1404],
        [ 4.3735, -4.3328]], device='cuda:0')
Classifier-guided loss: 0.00016210721514653414
Loss for all neutral texts: 0.00016210721514653414

Test Case 3: Mixed Texts
Calculating classifier-guided loss...
Logits: tensor([[-3.9838,  3.9090],
        [ 4.8206, -5.1404],
        [ 4.9345, -5.1655],
        [-3.9934,  3.9302]], device='cuda:0')
Classifier-guided loss: 3.954313278198242
Loss for mixed texts: 3.954313278198242



In [25]:
class Seq2SeqTrainerCustomLoss(Seq2SeqTrainer):
    def __init__(self, tokenizer_toxicity, model_toxicity, *args, **kwargs):
        super(Seq2SeqTrainerCustomLoss, self).__init__(*args, **kwargs)
        self.tokenizer_toxicity = tokenizer_toxicity
        self.model_toxicity = model_toxicity

    def prediction_step_custom(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
        **gen_kwargs,
    ) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
        """
        Perform an evaluation step on `model` using `inputs`.

        Subclass and override to inject custom behavior.

        Args:
            model (`nn.Module`):
                The model to evaluate.
            inputs (`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument `labels`. Check your model's documentation for all accepted arguments.
            prediction_loss_only (`bool`):
                Whether or not to return the loss only.
            gen_kwargs:
                Additional `generate` specific kwargs.

        Return:
            Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]: A tuple with the loss, logits and
            labels (each being optional).
        """

        if not self.args.predict_with_generate or prediction_loss_only:
            return super().prediction_step(
                model, inputs, prediction_loss_only=prediction_loss_only, ignore_keys=ignore_keys
            )

        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)

        # XXX: adapt synced_gpus for fairscale as well
        # Priority (handled in generate):
        # non-`None` gen_kwargs > model.generation_config > default GenerationConfig()
        if len(gen_kwargs) == 0 and hasattr(self, "_gen_kwargs"):
            gen_kwargs = self._gen_kwargs.copy()
        if "num_beams" in gen_kwargs and gen_kwargs["num_beams"] is None:
            gen_kwargs.pop("num_beams")
        if "max_length" in gen_kwargs and gen_kwargs["max_length"] is None:
            gen_kwargs.pop("max_length")

        gen_kwargs["synced_gpus"] = (
            gen_kwargs["synced_gpus"] if gen_kwargs.get("synced_gpus") is not None else False
        )

        generation_inputs = inputs.copy()
        # If the `decoder_input_ids` was created from `labels`, evict the former, so that the model can freely generate
        # (otherwise, it would continue generating from the padded `decoder_input_ids`)
        if (
            "labels" in generation_inputs
            and "decoder_input_ids" in generation_inputs
            and generation_inputs["labels"].shape == generation_inputs["decoder_input_ids"].shape
        ):
            generation_inputs = {k: v for k, v in inputs.items() if k != "decoder_input_ids"}
        generated_tokens = self.model.generate(**generation_inputs, **gen_kwargs)

        # Temporary hack to ensure the generation config is not initialized for each iteration of the evaluation loop
        # TODO: remove this hack when the legacy code that initializes generation_config from a model config is
        # removed in https://github.com/huggingface/transformers/blob/98d88b23f54e5a23e741833f1e973fdf600cc2c5/src/transformers/generation/utils.py#L1183
        if self.model.generation_config._from_model_config:
            self.model.generation_config._from_model_config = False

        # Retrieves GenerationConfig from model.generation_config
        gen_config = self.model.generation_config
        # in case the batch is shorter than max length, the output should be padded
        if generated_tokens.shape[-1] < gen_config.max_length:
            generated_tokens = self._pad_tensors_to_max_len(generated_tokens, gen_config.max_length)
        elif gen_config.max_new_tokens is not None and generated_tokens.shape[-1] < gen_config.max_new_tokens + 1:
            generated_tokens = self._pad_tensors_to_max_len(generated_tokens, gen_config.max_new_tokens + 1)

        if has_labels:
            with self.compute_loss_context_manager():
                outputs = model(**inputs)
            if self.label_smoother is not None:
                loss = self.label_smoother(outputs, inputs["labels"]).mean().detach()
            else:
                loss = (outputs["loss"] if isinstance(outputs, dict) else outputs[0]).mean().detach()
        else:
            loss = None

        if self.args.prediction_loss_only:
            return loss, None, None

        if has_labels:
            labels = inputs["labels"]
            if labels.shape[-1] < gen_config.max_length:
                labels = self._pad_tensors_to_max_len(labels, gen_config.max_length)
            elif gen_config.max_new_tokens is not None and labels.shape[-1] < gen_config.max_new_tokens + 1:
                labels = self._pad_tensors_to_max_len(labels, gen_config.max_new_tokens + 1)
        else:
            labels = None

        return loss, generated_tokens, labels

    def classifier_guided_loss(self, generated_text, target_label):
        # Tokenize the generated text
        inputs = self.tokenizer_toxicity(generated_text, return_tensors="pt", padding=True)
        
        # Move inputs to the same device as the model
        inputs = {key: tensor.to(DEVICE) for key, tensor in inputs.items()}
        
        # Get the batch size
        batch_size = inputs["input_ids"].shape[0]

        # Ensure target label is a tensor and on the correct device and shape
        target_label = torch.tensor([target_label]*batch_size, dtype=torch.long).to(DEVICE)

        # Get the classifier's output logits
        logits = self.model_toxicity(**inputs)["logits"]
        
        # Calculate the cross-entropy loss
        loss_fct = torch.nn.CrossEntropyLoss().to(DEVICE)
        loss = loss_fct(logits, target_label)
        
        return loss
    
    def compute_loss(self, model, inputs):
        """
        Compute custom loss for the model.

        Args:
            model (torch.nn.Module): The model training or evaluating.
            inputs (dict): The inputs and targets of the model.

        Returns:
            torch.FloatTensor: The loss value.
        """
        # # Get outputs, predictions and prediction loss
        # outputs = model(**inputs)
        # pred_loss = outputs.loss if isinstance(outputs, dict) else outputs[0]

        # Call prediction_step
        pred_loss, preds, refs = self.prediction_step_custom(model, inputs, prediction_loss_only=False)
        
        # Post-process the predictions and references
        ## In case the model returns more than the prediction logits
        if isinstance(preds, tuple):
            preds = preds[0]

        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)

        ## Replace -100s in the labels as we can't decode them
        refs = torch.where(refs != -100, refs, torch.tensor(self.tokenizer.pad_token_id).to(DEVICE))
        decoded_refs = self.tokenizer.batch_decode(refs, skip_special_tokens=True)

        ## Some simple post-processing
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_refs = [ref.strip() for ref in decoded_refs]

        # Calculate the classifier-guided loss
        classifier_loss = self.classifier_guided_loss(decoded_preds, 0)

        # Calculate total loss as a weighted sum of the classifier loss and the prediction loss
        total_loss = 0.5 * classifier_loss + 0.5 * pred_loss

        # Return total loss
        return total_loss

In [26]:

def setup_trainer_cl(output_dir_name,
                train_dataset,
                eval_dataset,
                model_checkpoint="t5-small",
                per_device_train_batch_size=32,
                per_device_eval_batch_size=128,
                learning_rate=1e-4,
                num_train_epochs=30,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=4,
                compute_metrics=compute_metrics,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
                tokenizer_toxicity=tokenizer_toxicity,
                model_toxicity=model_toxicity):
    """
    Set up a Seq2SeqTrainer object for training a T5 model with custom loss function

    Default parameters based on this: https://github.com/google-research/text-to-text-transfer-transformer/blob/main/t5/models/hf_model.py#L55

    Args:
        output_dir_name (str): What to name the model in the output directory.
        model_checkpoint (str): Name of the pre-trained model to use.
        train_dataset (Dataset): Training dataset.
        eval_dataset (Dataset): Validation/test dataset.
        per_device_train_batch_size (int): Batch size for training.
        per_device_eval_batch_size (int): Batch size for evaluation.
        learning_rate (float): Learning rate for optimizer.
        weight_decay (float): Weight decay for optimizer.
        num_train_epochs (int): Number of training epochs.
        max_length (int): Maximum length of generated sequences.
        num_beams (int): Number of beams for beam search.
        compute_metrics (function): Function to compute evaluation metrics.

    Returns:
        Seq2SeqTrainer: Trainer object for training the T5 model.
    """
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    GEN_CONFIG_PATH = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(GEN_CONFIG_PATH)

    # Define the training arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=GEN_CONFIG_PATH,
        fp16=True,
        report_to="wandb",
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
    )

    # Create a partial function with the tokenizer argument included
    compute_metrics_with_tokenizer = partial(compute_metrics, tokenizer=tokenizer)
    
    # Instantiate the trainer
    trainer = Seq2SeqTrainerCustomLoss(
        tokenizer_toxicity=tokenizer_toxicity,
        model_toxicity=model_toxicity,
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_with_tokenizer,
        callbacks=callbacks
    )

    return trainer

In [27]:
prefixed_datasets = add_prefix(raw_datasets)

tokenized_datasets_t5_small = prefixed_datasets.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small = setup_trainer_cl(
    output_dir_name="t5-small-detoxify-cl",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["validation"],
)

# wandb.init(project="w266_final_project", name="t5-small-detoxify-cl")
trainer_t5_small.train()
# wandb.finish()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

# Fine-Tune T5 Model (With Negative Word List)